# Aim of this notebook is to explore this dataset in much simple and clean way.

# **Import**

In [ ]:
import numpy as np 
import pandas as pd 
from statsmodels.graphics.gofplots import qqplot
import plotly.express as px
import seaborn as sns
from sklearn.model_selection import train_test_split
sns.set(style = "darkgrid")
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
%matplotlib inline

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

# **Read Dataset**

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-may-2022/sample_submission.csv")
train = pd.read_csv("../input/tabular-playground-series-may-2022/train.csv")
test = pd.read_csv("../input/tabular-playground-series-may-2022/test.csv")
print("Sample")
display(sample.head(2))
print()
print("Train")
display(train.head(2))
print()
print("Test")
display(test.head(2))

# **Overall view of dataset**
* train 900000 rows, test 700000 rows
* no nan values in our datasets
* 31 features
* f_00 - f_06 and f_19 - f_26 and f_28 := float columns
* f_07 - f_18 and f_29 - f_30 := int columns
* f_27 := object column
* target columns := binary (0/1) and target is almost balanced #0: 462161 and #1: 437839

In [ ]:
print("Sample, train, test")
print(sample.shape, train.shape, test.shape)
print()
print("No of null values")
print(sample.isnull().sum().sum(), train.isnull().sum().sum(), test.isnull().sum().sum())
print()
features = test.drop("id", axis=1).columns.tolist()
print(features)
print()
print(train.info())

In [ ]:
print(train.target.value_counts())
sns.countplot(x=train['target'])
plt.show()

# **Here we look at no of unique value in each columns**


* 700000+900000 = 1600000 [train+test]
* all float columns has different values in each row <br>
* all int columns has total at most 17 different unique values ( so these are some sort of categorical variables))<br>
* f_27 which is an object column(as has string entry) has total 1181880 unique values 160000-1181880=418120 repetitions
* In int columns there are many unique values whose frequency is less than 1%(see below). We can combine them to create new feature.

In [ ]:
full_data = pd.concat([train[features],test[features]], axis=0)
print(full_data.shape)
print()
list(zip(full_data.columns, full_data.dtypes, full_data.nunique()))

In [ ]:
cat_features = [i for i in features if full_data[i].nunique() <= 17]
num_features = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19', 'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']
print("features with no of unique values less than equal to 17")
print(cat_features)
print()
print("% of unique values")
for feat in cat_features:
    print(feat,":")
    a = full_data[feat].value_counts()*100/full_data.shape[0]
    print(a)
    print("="*40)
    print()

In [ ]:
print("f_27 :")
print(full_data.f_27.value_counts())

# **f_27**
**The f_27 column contains string of length 10 characters, Let's try to explore these encoding.**

We first created a new dataframe from f_27 by splitting these strings into 10 columns of each characters.<br>
We notice following things of this encoding:
* f0, f2, f5 : contains only two characters A,B  (can be used to create new features)
* f1, f3, f4, f6, f8, f9: all contains characters from A to O 
* f7: contains charactes from A to T
* f1, f3, f4, f6, f8, f9 : all has same distribution of characters 
* except f7 which has almost same frequency of each character

In [ ]:
data_f_27 = pd.DataFrame([list(i) for i in sorted(full_data.f_27.value_counts().index.values)])
data_f_27.columns = ["f0","f1","f2","f3","f4","f5","f6","f7","f8","f9"]
data_f_27.head(3)

In [ ]:
for i in range(10):
    print(data_f_27.groupby([f"f{i}"]).count().iloc[:,0])
    print("="*40)
    print()


* f0 and f5 have very similar distribution while f2 has just opposite distribution

In [ ]:
plt.figure(figsize=(12,6))
for i in [0,2,5]:
    d= data_f_27[f"f{i}"].value_counts()
    plt.plot(d,label=f"f{i}")
plt.legend()
plt.show()
plt.figure(figsize=(12,6))
for i in [1,3,4,6,7,8,9]:
    d= data_f_27[f"f{i}"].value_counts()
    plt.plot(d, label=f"f{i}")
plt.legend()
plt.show()

<code>px.treemap()</code> is used to visualize proportions for multiple columns at at time.

In [ ]:
fig  = px.treemap(data_f_27.sample(20), path= data_f_27.columns.tolist() ) 
fig.show()

* Parallel Sets represents contribution of columns on each other. <br>
* It is used to represent inter-connection among columns.<br>
* Note: It works only for Object and int data type columns.<br>
* We can set color value based on a column which can be int/float type.
> We have created two plots:-
1. In first plot we have taken full_data i.e. train+test 
1. In second plot we have taken only train data with target as color

In [ ]:
px.parallel_categories(data_f_27.sample(200)) # train+test

In [ ]:
train_f_27= pd.DataFrame([list(i) for i in train.f_27.value_counts().index.values])
train_f_27.columns = ["f0","f1","f2","f3","f4","f5","f6","f7","f8","f9"]
train_f_27["target"] = train.target
train_f_27.head(3)

In [ ]:
px.parallel_categories(train_f_27.head(800),color="target") # train

# features = cat_features + num_features + f_27
* cat_features := 14
* num_features := 16
* f_27

In [ ]:
display(full_data[cat_features].head(2))

display(full_data[num_features].head(2))

display(full_data[["f_27"]].head(2))

# SHAP
Most of the ideas below are inspired from @ambrosm and @wti200 work.
Link of the notebooks: 

https://www.kaggle.com/code/wti200/analysing-interactions-with-shap

https://www.kaggle.com/code/ambrosm/tpsmay22-eda-which-makes-sense/notebook

In [ ]:
# From https://www.kaggle.com/ambrosm/tpsmay22-eda-which-makes-sense
for df in [train, test]:
    for i in range(10):
        df[f'ch{i}'] = df.f_27.str.get(i).apply(ord) - ord('A')
    df["unique_characters"] = df.f_27.apply(lambda s: len(set(s)))

In [ ]:
test.head(2)

In [ ]:
features = [col for col in train.columns if col != "target" and col !="f_27" and col != "id"]
print(features)

In [ ]:
import shap

In [ ]:
# Here we have to create shap_interaction matrix. Since this takes much time so we will take it other notebook. Thanks to @wti200 for making it public :-))

# xtr = train[features]
# xte = test[features]
# X_train, X_val, y_train, y_val = train_test_split(xtr,train.target, test_size=0.4, random_state = 42)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)

# from lightgbm import LGBMClassifier
# from sklearn.metrics import roc_auc_score
# # Train model
# lgbm_model =LGBMClassifier(n_estimators=5000, min_child_samples=80, random_state=1307)
# lgbm_model.fit(X_train.values, y_train)
# y_val_pred = lgbm_model.predict_proba(X_val.values)[:,1]
# score = roc_auc_score(y_val, y_val_pred)
# print(f"Validation AUC:{(score):.3f}")

# # Using a random sample of the dataframe for better time computation
# X_sampled = X_val.sample(20000, random_state=1307)
# X_sampled.shape

# # explain the model's predictions using SHAP values
# explainer = shap.TreeExplainer(lgbm_model)
# shap_values = explainer.shap_values(X_sampled)
# print(shap_values[0].shape, shap_values[1].shape, len(shap_values))

In [ ]:
#Get SHAP interaction values. Beware it is time consuming to calculate the interaction values.
# shap_interaction = explainer.shap_interaction_values(X_sampled)
# print(np.shape(shap_interaction))

loaded_arr = np.loadtxt('../input/tpsmay-shap/shap_interaction_20k.txt')
load_original_arr = loaded_arr.reshape(
    #loaded_arr.shape[0], loaded_arr.shape[1] // shap_interaction.shape[2], shap_interaction.shape[2])
    loaded_arr.shape[0], loaded_arr.shape[1] // 41, 41)

shap_interaction = load_original_arr
print(np.shape(shap_interaction))

# SHAP values are used to explain individual predictions made by a model.
print(shap_interaction[0].shape)

mean_shap = np.abs(shap_interaction).mean(0)
print(mean_shap.shape)

In [ ]:
sns.heatmap(shap_interaction[0])
plt.show()
sns.heatmap(mean_shap)
plt.show()

In [ ]:
df = pd.DataFrame(mean_shap, index=features, columns=features)
df.head(2)

In [ ]:
plt.figure(figsize=(15,10 ))
sns.heatmap(df)

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df.where(df.values == np.diagonal(df)))

In [ ]:
"""
df.where( cond, 3) := fill by 3 where cond is FALSE
df.mask(cond, 3) := fill by 3 where cond is TRUE
"""
df1= df.where(df.values == np.diagonal(df), 2*df.values) # increase all non diagonal elements by factor of 2
fig= plt.figure(figsize=(35, 20), facecolor='#002637', edgecolor='r')
ax = fig.add_subplot()
sns.heatmap(df1.round(decimals=3), cmap="coolwarm", annot=True, fmt=".6g", cbar=False, ax=ax)
ax.tick_params(axis='x', colors='w', labelsize=15, rotation=90)
ax.tick_params(axis='y', colors='w', labelsize=15)

plt.suptitle("SHAP interaction values", color="white", fontsize=60, y=0.97)
plt.yticks(rotation=0) 
plt.show()

In [ ]:
# threshold

df1= df.where(df.values == np.diagonal(df), 2*df.values)

df1 = df1.where(df1.values >= .4)
fig= plt.figure(figsize=(30, 30), facecolor='#002637', edgecolor='r')
ax = fig.add_subplot()
sns.heatmap(df1.round(decimals=2), cmap="coolwarm", annot=True, fmt=".6g", cbar=False, ax=ax, linewidths=.2, mask= np.triu(df1)) # triu: tril
ax.tick_params(axis='x', colors='w', labelsize=15, rotation=90)
ax.tick_params(axis='y', colors='w', labelsize=15)

plt.suptitle("SHAP interaction values", color="white", fontsize=60, y=0.97)
plt.yticks(rotation=0) 
plt.show()

> f_00 and f_26

> f_01 and f_26

> f_02 and f_21 

> f_05 and f_22 

> f_24 adn f_30 

> f_25 and f_30 

> f_26 and f_30

We will further verify these interaction.

In [ ]:
interaction_cols= ["f_00", "f_01", "f_02", "f_05", "f_21","f_22", "f_24", "f_25" ,"f_26", "f_30"]
interaction_cols

In [ ]:
len(interaction_cols)

In [ ]:
# We manually check if there is any relation between features
fig,axes  = plt.subplots(9,5,figsize=(30,30))
flatten_axes = axes.flatten()
counter = 0
for i in range(len(interaction_cols)):
    for j in range(len(interaction_cols)):
        if i>j:
            c1 = interaction_cols[i]
            c2 = interaction_cols[j]
            cmap = ListedColormap(["#ffd700", "#0057b8"])
            flatten_axes[counter].scatter(train[c1], train[c2], s=1,c=train.target, cmap=cmap,)
            flatten_axes[counter].set_xlabel(c1)
            flatten_axes[counter].set_ylabel(c2)
            #flatten_axes[counter].set_aspect('equal')
            counter += 1
plt.show()

In [ ]:
# finds all possible selection possible
col_list = ["f_00","f_01","f_02","f_05"]
new_list = []
for i in range(16):
    val = int(bin(i)[2:])
    temp_list = []
    counter = 1
    while val != 0:
        if val %10 == 1:
            # take it
            temp_list.append(col_list[-counter])
        val = val//10 
        counter += 1
    new_list.append(temp_list)

In [ ]:
new_list, len(new_list)

In [ ]:
for ls in new_list:
    if len(ls) == 0:
        continue 
    print(ls)
    val= train[ls].sum(axis=1).values
    cmap = ListedColormap(["#ffd700", "#0057b8"])
    plt.scatter(val ,train['f_21'], s=1,c=train.target, cmap=cmap)
    plt.show()

In [ ]:
cmap = ListedColormap(["#ffd700", "#0057b8"])
plt.scatter(train['f_00'] + train['f_01'] ,train['f_26'], s=1,c=train.target, cmap=cmap)

### Althoug f_02 and f_21 shows high correlation but other featues like f_01, f_05, f_00 also show significant correlation with f_21 :
### We will create features and test models to see which works for us.

> f_00 + f_01  f_26 

> f_05  f_22 

> f_02 f_21 

> ---------

> f_00 f_21 

> f_01 f_21 

> f_05 f_21

In [ ]:
train.columns

In [ ]:
for df in [train, test]:
    df['i_02_21'] = (df.f_21 + df.f_02 > 5.2).astype(int) - (df.f_21 + df.f_02 < -5.3).astype(int)
    df['i_05_22'] = (df.f_22 + df.f_05 > 5.1).astype(int) - (df.f_22 + df.f_05 < -5.4).astype(int)
    i_00_01_26 = df.f_00 + df.f_01 + df.f_26
    df['i_00_01_26'] = (i_00_01_26 > 5.0).astype(int) - (i_00_01_26 < -5.0).astype(int)

# Feature correlation
* correlation between numerical features <b>[ there is some correlation between (f_28, f_2) (f_28, f_3) (f_28, f_5) and (f_25, f_23) ]</b>
* correlation between categorical features <b>[ there is no correlation among categorical features ]</b>

In [ ]:
feat = num_features 
fig, ax = plt.subplots(1,2,figsize=(32,11))         # Sample figsize in inches
ax[0].title.set_text("train")
ax[1].title.set_text("test")
sns.heatmap(train[feat + ['target']].corr().abs(), cmap="viridis", linewidths=.5, ax=ax[0], annot=True, fmt=".2f")
sns.heatmap(test[feat].corr().abs(), cmap="viridis",linewidths=.5, ax=ax[1], annot=True, fmt=".2f")
plt.show()

## threshold of .2
fig, ax = plt.subplots(1,2,figsize=(32,11))         # Sample figsize in inches
ax[0].title.set_text("train")
ax[1].title.set_text("test")
sns.heatmap(train[feat+ ['target']].corr().abs()>.2, cmap="coolwarm", linewidths=.5, ax=ax[0],annot=True, fmt=".2f")
sns.heatmap(test[feat].corr().abs()>.2, cmap="coolwarm",linewidths=.5, ax=ax[1],annot=True, fmt=".2f")
plt.show()

In [ ]:
feat = cat_features 
fig, ax = plt.subplots(1,2,figsize=(32,11))         
ax[0].title.set_text("train")
ax[1].title.set_text("test")
sns.heatmap(train[feat+ ['target']].corr().abs(), cmap="viridis", linewidths=.5, ax=ax[0], annot=True, fmt=".2f")
sns.heatmap(test[feat].corr().abs(), cmap="viridis",linewidths=.5, ax=ax[1], annot=True, fmt=".2f")
plt.show()
## threshold of .2
fig, ax = plt.subplots(1,2,figsize=(32,11))         
ax[0].title.set_text("train")
ax[1].title.set_text("test")
sns.heatmap(train[feat+ ['target']].corr().abs()>.2, cmap="coolwarm", linewidths=.5, ax=ax[0],annot=True, fmt=".2f")
sns.heatmap(test[feat].corr().abs()>.2, cmap="coolwarm",linewidths=.5, ax=ax[1],annot=True, fmt=".2f")
plt.show()

> Drawback: It only shows linear dependency between target and other features.

> No feature is Strongly correlated with target.

If we plot a rolling mean of the target probability for every feature, we'll see nonlinear dependences as well. A horizontal line means that the target does not depend on the feature (e.g., f_03, f_04, f_06), a line with low minimum and high maximum shows a high mutual information between feature and target (e.g., f_19, f_21, f_28). Credit: @ambrosm

Proceidure: 
For each feature, we sort target by feature value. 
Then we take rolling mean of target and assign it to corresponding data point. 
Then we scatter plot feature vs rolling mean 

In [ ]:
from cycler import cycler
plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])

In [ ]:
# Plot dependence between every feature and the target
def plot_mutual_info_diagram(df, features, ncols=4, by_quantile=True, mutual_info=True,
                             title='How the target probability depends on single features'):
    def H(p):
        """Entropy of a binary random variable in nat"""
        # Entropy means randomness
        return -np.log(p) * p - np.log(1-p) * (1-p)
                 
    nrows = (len(features) + ncols - 1) // ncols
    fig, axs = plt.subplots(nrows, ncols, figsize=(16, nrows*4), sharey=True)
    for f, ax in zip(features, axs.ravel()):
        temp = pd.DataFrame({f: df[f].values,
                             'state': df.target.values})
        temp = temp.sort_values(f)
        temp.reset_index(inplace=True)
        rolling_mean = temp.state.rolling(15000, center=True, min_periods=1).mean()
        if by_quantile:
            ax.scatter(temp.index, rolling_mean, s=2)
        else:
            ax.scatter(temp[f], rolling_mean, s=2)
        if mutual_info and by_quantile:
            # entropy of target_mean - mean( entropy of rolling means)
            ax.set_xlabel(f'{f} mi={H(temp.state.mean()) - H(rolling_mean[~rolling_mean.isna()].values).mean():.5f}')
        else:
            ax.set_xlabel(f'{f}')
    plt.suptitle(title, y=0.90, fontsize=20)
    plt.show()

plot_mutual_info_diagram(train, num_features,
                         title='How the target probability depends on the float features')

> There are many non linear relationship

In [ ]:
plot_mutual_info_diagram(train, cat_features,
                         title='How the target probability depends on the float features')

## Categorial features(features which has no of unique values less than equal to 17)
* both train and test set have same distribution 
* both train and test set don't follow normal distribution

Q-Q plot also known as (Quantile-Quantile plot) is used to check whether our data follows normal distribution or not.
If our plot lies on the red line(y=x) then it is normally distributed. It it don't lie on the y=x line then our feature is not normally distributed.

In [ ]:
print("histplot"," "*3,"Kde plot"," "*3, "Boxplot"," "*3,"QQplot train"," "*3,"QQplot test")
fig, axes = plt.subplots(14,5, figsize=(25,60))
axes = axes.flatten()
for i in range(0,len(axes),5):
    col = cat_features[i//5]
    ax = axes[i]
    train[col].hist(ax= ax,bins=20, color="r",alpha=.5, label="train")
    test[col].hist(ax= ax,bins=20, color="b", alpha=.5, label="test")
    
    sns.kdeplot(train[col], color="red", label="train", ax=axes[i+1])
    sns.kdeplot(test[col],  color="green", label="test", ax=axes[i+1])
    axes[i+1].legend()
    
    sns.boxplot(data=train[col], color="red",ax=axes[i+2])
    sns.boxplot(data= test[col],  color="green", ax=axes[i+2])
    axes[i+2].legend() 
    
    t1= (train[col].values - train[col].values.mean())/ train[col].values.std()
    t2= (test[col].values - test[col].values.mean())/ test[col].values.std()
    qqplot(t1,line="s",ax=axes[i+3])
    qqplot(t2,line="s",ax=axes[i+4])
    ax.get_yaxis().set_visible(False)
    ax.set_title(f'f{cat_features[i//5]}', loc = 'right', fontsize = 12)
    ax.legend()
    fig.suptitle("distribution of train-test cat_features")
    fig.tight_layout()  
plt.show()

## numerical features
* both train and test set have same distribution 
* both train and test set follow normal distribution <b>[with slight deviation from normal behaviour for f_25 and f_26]</b>

In [ ]:
print("histplot"," "*3,"Kde plot"," "*3, "Boxplot"," "*3,"QQplot train"," "*3,"QQplot test")
fig, axes = plt.subplots(16,5, figsize=(25,70))
axes = axes.flatten()
for i in range(0,len(axes),5):
    col = num_features[i//5]
    ax = axes[i]
    train[col].hist(ax= ax,bins=20, color="r",alpha=.5, label="train")
    test[col].hist(ax= ax,bins=20, color="b", alpha=.5, label="test")

    sns.kdeplot(train[col], color="red", label="train", ax=axes[i+1])
    sns.kdeplot(test[col],  color="green", label="test", ax=axes[i+1])
    axes[i+1].legend()
    
    sns.boxplot(data=train[col], color="red",ax=axes[i+2])
    sns.boxplot(data= test[col],  color="green", ax=axes[i+2])
    axes[i+2].legend()    
    
    t1= (train[col].values - train[col].values.mean())/ train[col].values.std()
    t2= (test[col].values - test[col].values.mean())/ test[col].values.std()
    qqplot(t1,line="s",ax=axes[i+3])
    qqplot(t2,line="s",ax=axes[i+4])
    ax.get_yaxis().set_visible(False)
    ax.set_title(f'{num_features[i//5]}', loc = 'right', fontsize = 12)
    ax.legend()
    fig.suptitle("distribution of train-test num_features")
    fig.tight_layout()   
plt.show()

## Now for numerical columns we will see if there is any outlier, if present then we will remove it.

In [ ]:
def check_outlier(data,col_name):
    """
    input:= data, column name
    output:= Lower wishker and Upper wishker 
    """
    Q3 = data[col_name].quantile(0.75)
    Q1 = data[col_name].quantile(0.25)
    IQR = Q3-Q1 
    print("75%:", Q3)
    print("25%",Q1)
    print("IQR:",IQR)
    
    LW = Q1 - 1.5*IQR 
    UW = Q3 + 1.5*IQR 
    print("Lower and Upper Wishker: ",LW, UW)
    print("Min and Max value: ", np.min(data[col_name]),np.max(data[col_name]))
    print("Full data:", data.shape)
    print("No of outliers: ",data[(data[col_name]<LW) | (data[col_name]>UW)].shape)
    
    sns.boxplot(x=data[col_name])
    sns.stripplot(x=data[col_name], color="0.5")
    plt.show()
    return LW, UW
    

In [ ]:
for c in num_features:
    print("Column: ",c)
    LW, UW= check_outlier(train,c)
    print("After removing outliers")
    train=train[(train[c]>= LW) & (train[c]<= UW)]
    sns.boxplot(x=train[c])
    sns.stripplot(x=train[c], color="0.5")
    plt.show()
    print("="*40)

## Plot after removing outliers from train set.
* We can see that, now our train set is not following normal distribution in tail region, because we have removed outliers(from tails). But now our dataset is much more stable.

In [ ]:
print("histplot"," "*3,"Kde plot"," "*3, "Boxplot"," "*3,"QQplot train"," "*3,"QQplot test")
fig, axes = plt.subplots(16,5, figsize=(25,70))
axes = axes.flatten()
for i in range(0,len(axes),5):
    col = num_features[i//5]
    ax = axes[i]
    train[col].hist(ax= ax,bins=20, color="r",alpha=.5, label="train")
    test[col].hist(ax= ax,bins=20, color="b", alpha=.5, label="test")

    sns.kdeplot(train[col], color="red", label="train", ax=axes[i+1])
    sns.kdeplot(test[col],  color="green", label="test", ax=axes[i+1])
    axes[i+1].legend()
    
    sns.boxplot(data=train[col], color="red",ax=axes[i+2])
    sns.boxplot(data= test[col],  color="green", ax=axes[i+2])
    axes[i+2].legend()    
    
    t1= (train[col].values - train[col].values.mean())/ train[col].values.std()
    t2= (test[col].values - test[col].values.mean())/ test[col].values.std()
    qqplot(t1,line="s",ax=axes[i+3])
    qqplot(t2,line="s",ax=axes[i+4])
    ax.get_yaxis().set_visible(False)
    ax.set_title(f'{num_features[i//5]}', loc = 'right', fontsize = 12)
    ax.legend()
    fig.suptitle("distribution of train-test num_features")
    fig.tight_layout()   
plt.show()

## Let's check now the correlations

In [ ]:
for ls in new_list:
    if len(ls) == 0:
        continue 
    print(ls)
    val= train[ls].sum(axis=1).values
    cmap = ListedColormap(["#ffd700", "#0057b8"])
    plt.scatter(val ,train['f_21'], s=1,c=train.target, cmap=cmap)
    plt.show()

In [ ]:
# We manually check if there is any relation between features
fig,axes  = plt.subplots(9,5,figsize=(30,30))
flatten_axes = axes.flatten()
counter = 0
for i in range(len(interaction_cols)):
    for j in range(len(interaction_cols)):
        if i>j:
            c1 = interaction_cols[i]
            c2 = interaction_cols[j]
            cmap = ListedColormap(["#ffd700", "#0057b8"])
            flatten_axes[counter].scatter(train[c1], train[c2], s=1,c=train.target, cmap=cmap,)
            flatten_axes[counter].set_xlabel(c1)
            flatten_axes[counter].set_ylabel(c2)
            #flatten_axes[counter].set_aspect('equal')
            counter += 1
plt.show()

# If you like my work please do upvote!
**<span style="color:#444160;"> Thanks!🙂</span>**<br>
.<br>
.<br>
.

<img src="https://media.giphy.com/media/SfYTJuxdAbsVW/giphy.gif" width=70%>